# Writing a SAL Script

This notebooks contains the example used for development of a SAL Script from a notebook.

In this example we will perform a dithering pattern on the sky with the Auxiliary Telescope and take a sequence of images and each position.

We will use functionality from the two main observatory control classes for the Auxiliary Telescope.
More information about these can be found in the [ts_observatory_control user guide](https://ts-observatory-control.lsst.io/user-guide/user-guide.html).

In [ ]:
import asyncio
import logging

import numpy as np
from matplotlib import pyplot as plt

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils.enums import RotType

## Setting up logging

When running on a notebook you may be interested in getting logging feedback. To enable this you may want to setup Python logging facility.

This next cell will setup the basic log configuration in debug mode.
If you find this too verbose and want to change the level, you can replace `logging.DEBUG` with `logging.INFO`, `logging.WARNING`, `logging.ERROR` or skip the next cell altogether. 

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

Matplotlib can be chatty so, better increase its log level.

In [ ]:
logging.getLogger("matplotlib").setLevel(logging.WARNING)

## Setup control classes

The first step in interacting with the system is to setup the SalObj library and the control classes.

This is done by creating a `salobj.Domain`, an object to handle the DDS communication and later passing it to the control classes.

In [ ]:
domain = salobj.Domain()

In [ ]:
atcs = ATCS(domain)

In [ ]:
latiss = LATISS(domain)

### Reducing salobj.Remote internal logging.

The internal `salobj.Remote` classes can get very chatty due to the incomming traffic from the CSCs.
You can reduce this by using a method provided by the control classes.

In [ ]:
atcs.set_rem_loglevel(logging.ERROR)

In [ ]:
latiss.set_rem_loglevel(logging.ERROR)

### Wait for salobj to finish setup DDS communication.

This is a background task that we need to `await` before we can communicate with the components. 

In [ ]:
await asyncio.gather(atcs.start_task, latiss.start_task)

## Executing operation

From now on we are ready to interact with the system.

We are now going to write down the loop that performs the dithering and data taking.

I will assume you had some time to think about the problem and exercice it enough to get confortable with parameterizing it and so on.

The idea is to develop a procedure that will do the following:

1. Slew to a target that is defined by a name that can be [resolved by simbad](http://simbad.u-strasbg.fr/simbad/sim-fid), and a [rotator setup](https://ts-observatory-control.lsst.io/user-guide/tcs-user-guide-generic.html#rotator-position-and-sky-position-angle).

2. Given a pre-defined grid of x/y offsets from the original position;
   1. Offset the telescope to each,
   2. Take a set of pre-defined observations.

We start by defining the parameters in the cells bellow.

### Target definition

The next cell defines the target to slew to and the rotator value/type.

In [ ]:
target_name = "HD 164461"
rot_value = 0.
rot_type = RotType.PhysicalSky

### Define offset grid

In [ ]:
grid_x = (np.random.rand(11)-0.5)*120.  # offset in image coordinate x-axis (in arcsec)
grid_y = (np.random.rand(11)-0.5)*120.  # offset in image coordinate y-axis (in arcsec)

We are in a Jupyter notebook so we might as well plot the grid generated above.

In [ ]:
plt.plot(grid_x, grid_y, '.:')

plt.xlabel("x-offset in arcsec")
plt.ylabel("y-offset in arcsec")

### Define observations setup

In [ ]:
exptime = [5., 10., 20.]  # list of exposure times in seconds
obs_filter = ["RG610", "RG610", "RG610"]  # list of filters 
obs_grating = ["empty_1", "ronchi90lpmm", "ronchi170lpmm"]  # list of gratings

### Run observation sequence

Now we have the parameters defined we can run a loop that will execute the dithering and observing sequence.

In [ ]:
await atcs.slew_object(name=target_name, rot=rot_value, rot_type=rot_type)

In [ ]:
for xx, yy in zip(grid_x, grid_y):
    # Offset telescope
    # Use non-relative offset as they are easier to reset
    await atcs.offset_xy(x=xx, y=yy, relative=False)
    
    # Take data
    for etime, flt, grt in zip(exptime, obs_filter, obs_grating):
        await latiss.take_object(exptime=etime, filter=flt, grating=grt)

In [ ]:
# Reset offset
await atcs.offset_xy(x=0., y=0., relative=False)